In [8]:
import pickle
import re
import pandas as pd
import os

Try and gather the s-box info from sage. If it doesn't already exist, generate it.

In [9]:
try: 
    with open("../chipwhisperer_minimal/generate_c/sboxes_info.pkl", "rb") as f:
        sboxes_df = pickle.load(f)
except FileNotFoundError:
    print("No pickled sbox info found, generating by running sboxes_info.py")
    import sboxes_info
    sboxes_info.main()
    with open("../chipwhisperer_minimal/generate_c/sboxes_info.pkl", "rb") as f:
        sboxes_df = pickle.load(f)

Get all results files (.txt files)

In [12]:
files = []
for filename in os.scandir("./"):
    if filename.is_file():
        # Only grab the txt files
        if re.search(r'\.txt$', filename.name):
            files.append(filename.name)
files

['n_traces_cwlitearm_DPA.txt',
 'avg_leaks_cwlitearm_CBC.txt',
 'n_traces_cwnano_DPA.txt',
 'avg_leaks_cwnano_CTR.txt',
 'avg_leaks_cwlitearm_CTR.txt',
 'n_traces_cwlitearm_CPA.txt',
 'avg_leaks_cwlitearm_ECB.txt',
 'avg_leaks_cwnano_CBC.txt',
 'avg_leaks_cwnano_ECB.txt',
 'n_traces_cwnano_CPA.txt']

Get associated column names for each result

In [19]:
col_names = ["" for _ in range(len(files))]

regex = r'(n_traces|avg_leaks)_(cwnano|cwlitearm)_(DPA|CPA|ECB|CBC|CTR)'
for i, filename in enumerate(files):
    # Get captured patterns
    matches = re.findall(regex, filename)[0]
    # Replace empty strings with None
    matches = [None if x == '' else x for x in matches]
    print(matches)

    # Make associated column names
    metric = matches[0]
    device = matches[1]
    metric_config = matches[2]

    col_name = f"{device}_{metric_config}_{metric}"

    col_names[i] = col_name

['n_traces', 'cwlitearm', 'DPA']
['avg_leaks', 'cwlitearm', 'CBC']
['n_traces', 'cwnano', 'DPA']
['avg_leaks', 'cwnano', 'CTR']
['avg_leaks', 'cwlitearm', 'CTR']
['n_traces', 'cwlitearm', 'CPA']
['avg_leaks', 'cwlitearm', 'ECB']
['avg_leaks', 'cwnano', 'CBC']
['avg_leaks', 'cwnano', 'ECB']
['n_traces', 'cwnano', 'CPA']


Import Results

In [20]:
sboxes_df.pop("box")
sboxes_df.pop("inverse")

sboxes_dict = sboxes_df.T.to_dict()
for i, filename in enumerate(files):
    col_name = col_names[i]
    with open(filename, "r") as f:
        lines = f.readlines()
        for line in lines:
            vals = re.split(r'\s', line)
            name = vals[0]
            result = vals[1]
            sboxes_dict[name][col_name] = result

sboxes_info_df = pd.DataFrame.from_dict(sboxes_dict).T

Round columns

In [21]:
sboxes_info_df = sboxes_info_df.apply(pd.to_numeric)
sboxes_info_df = sboxes_info_df.round(5)

Print to CSV

In [22]:
sboxes_info_df.to_csv("./sboxes_results.csv")